In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import * 
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.classification import RandomForestClassifier

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1615313233588_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
ss = SparkSession.builder.config("spark.executor.memory","20g")\
                         .config("spark.driver.memory","20g")\
                         .config("spark.executor.cores",6)\
                         .config('spark.executor.instances',8)\
                         .config("spark.driver.extraClassPath", "lib/sparknlp.jar")\
                         .config("spark.driver.maxResultSize",0)\
                         .config("spark.dynamicAllocation.enabled","false")\
                         .getOrCreate()
sc = ss.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
final_df=ss.read.parquet("s3a://facebookmsds694/final_df.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
final_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|          Text_array|               Title|                Body|                Tags|      Text_array_new|        Text_Vectors|         Tags_joined|count|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|use,condit,given,...|how to use a whil...|<p>i am fetching ...|       php mysql sql|[use, condit, giv...|(20000,[0,1,9,22,...|       php mysql sql| 1649|
|lose,day,random,a...|I lose each day a...|<p>Ok my problem ...|         probability|[lose, day, rando...|(20000,[19,20,30,...|         probability| 3847|
|regard,painleve',...|Regarding Painlev...|<p>Painleve's fir...|    complex-analysis|[regard, painleve...|(20000,[2,66,73,9...|    complex-analysis| 2911|
|contentel,part,vi...|Why ContentElemen...|<p>I was wonderin...|      

In [5]:
filtered_df = final_df.withColumn("num_labels",size(split(col('Tags')," ").cast("array<string>"))).filter(col("num_labels")==1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
filtered_df_cache=filtered_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
filtered_df_cache.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

617133

In [8]:
c='Tags'
si = StringIndexer(inputCol=c, outputCol=c + "-num")
sm = si.fit(filtered_df_cache)
newdf = sm.transform(filtered_df_cache).drop(c)
newdf = newdf.withColumnRenamed(c + "-num", c)
newdf.cache()
newdf.show()
filtered_df_cache.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|          Text_array|               Title|                Body|      Text_array_new|        Text_Vectors|         Tags_joined|count|num_labels| Tags|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|lose,day,random,a...|I lose each day a...|<p>Ok my problem ...|[lose, day, rando...|(20000,[19,20,30,...|         probability| 3847|         1| 25.0|
|regard,painleve',...|Regarding Painlev...|<p>Painleve's fir...|[regard, painleve...|(20000,[2,66,73,9...|    complex-analysis| 2911|         1| 35.0|
|contentel,part,vi...|Why ContentElemen...|<p>I was wonderin...|[contentel, part,...|(20000,[0,1,6,41,...|                 wpf| 2674|         1| 39.0|
|custom,cell,colum...|custom cell and c...|<p>Im using a cus...|[custom, cell, co...|(20000,[0

In [9]:
sets = newdf.randomSplit([0.8, 0.2], seed = 100)
train = sets[0].cache()
valid = sets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
train.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|          Text_array|               Title|                Body|      Text_array_new|        Text_Vectors|         Tags_joined|count|num_labels| Tags|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|
,possibl,duplica...|<?= what does thi...|<blockquote>
  <p...|[
, possibl, dupl...|(20000,[0,18,53,6...|                 c++|15328|         1|  8.0|
|"!!",oper,javascr...|What is the "!!" ...|<blockquote>
  <p...|["!!", oper, java...|(20000,[0,1,4,8,1...|          javascript|24995|         1|  3.0|
|"".join(reversed(...|"".join(reversed(...|<p>So according t...|["".join(reversed...|(20000,[50,139,22...|              python|16470|         1|  6.0|
|"#select,option:s...|$("#select option...|<p>I am able to k...|["#select, option...|(20000,[0

In [11]:
valid.show()
newdf.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|          Text_array|               Title|                Body|      Text_array_new|        Text_Vectors|         Tags_joined|count|num_labels| Tags|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+-----+
|"$$q",mean,perl,s...|What does "$$q" m...|<p>I was studying...|["$$q", mean, per...|(20000,[10,31,32,...|                perl| 4070|         1| 23.0|
|"&",i.e,symbol,an...|How to have "&" i...|<p>I want to have...|["&", i.e, symbol...|(20000,[8,9,23,61...|             android|49783|         1|  0.0|
|"+","@","*",mean,...|What do "+", "@" ...|<p>On the iReport...|["+", "@", "*", m...|(20000,[0,100,127...|               xpath|  526|         1|203.0|
|".#filename",emac...|What is ".#FILENA...|<blockquote>
  <p...|[".#filename", em...|(20000,[0

In [12]:
train=train.select(col("Text_Vectors").alias("features"),col("Tags").alias("label"),(col("count")).alias("weights"))
valid=valid.select(col("Text_Vectors").alias("features"),col("Tags").alias("label"),(col("count")).alias("weights"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
train.unpersist()
train.cache()
train.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-------+
|            features|label|weights|
+--------------------+-----+-------+
|(20000,[0,18,53,6...|  8.0|  15328|
|(20000,[0,1,4,8,1...|  3.0|  24995|
|(20000,[50,139,22...|  6.0|  16470|
|(20000,[0,23,29,6...|  2.0|  28079|
|(20000,[8,62,70,7...|102.0|   1183|
|(20000,[7,20,23,4...|  6.0|  16470|
|(20000,[0,6,57,69...|  4.0|  23751|
|(20000,[1,5,6,36,...|308.0|    349|
|(20000,[0,11,17,1...| 40.0|   2616|
|(20000,[0,4,8,10,...|  2.0|  28079|
|(20000,[1,2,3,5,8...|  3.0|  24995|
|(20000,[0,1,5,10,...|  5.0|  22889|
|(20000,[76,248,25...|181.0|    635|
|(20000,[0,4,6,8,1...|  0.0|  49783|
|(20000,[1,8,12,20...|178.0|    651|
|(20000,[0,15,21,3...| 46.0|   2153|
|(20000,[0,1,4,5,6...|231.0|    450|
|(20000,[2,4,6,7,2...| 68.0|   1704|
|(20000,[60,320,70...|210.0|    510|
|(20000,[2,3,7,15,...| 49.0|   2077|
+--------------------+-----+-------+
only showing top 20 rows

In [14]:
valid.unpersist()
valid.cache()
valid.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-------+
|            features|label|weights|
+--------------------+-----+-------+
|(20000,[10,31,32,...| 23.0|   4070|
|(20000,[8,9,23,61...|  0.0|  49783|
|(20000,[0,100,127...|203.0|    526|
|(20000,[0,3,7,19,...| 72.0|   1605|
|(20000,[0,1,5,6,1...| 24.0|   4010|
|(20000,[26,29,35,...|274.0|    385|
|(20000,[0,69,97,2...|223.0|    458|
|(20000,[0,6,39,16...|  6.0|  16470|
|(20000,[0,26,42,4...|180.0|    639|
|(20000,[1,5,8,14,...|297.0|    362|
|(20000,[1,75,195,...|  4.0|  23751|
|(20000,[0,1,2,7,8...| 13.0|   8479|
|(20000,[0,1,10,15...| 57.0|   1872|
|(20000,[0,4,5,9,2...| 67.0|   1707|
|(20000,[1,6,12,19...|229.0|    455|
|(20000,[1,7,23,24...|127.0|    864|
|(20000,[0,49,50,1...|190.0|    580|
|(20000,[0,1,3,4,7...|  4.0|  23751|
|(20000,[5,6,7,8,1...| 63.0|   1774|
|(20000,[0,79,89,9...| 57.0|   1872|
+--------------------+-----+-------+
only showing top 20 rows

## Naive Bayes

In [ ]:
## Shruti Roy 
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=0.2)
model=nb.fit(train)
Nbpredict=model.transform(valid)

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(Nbpredict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6702920044995749

## Weighted Naive Bayes 

In [17]:
## Aneri Dand
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=0.2,weightCol="weights")
model=nb.fit(train)
Nbpredict=model.transform(valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(Nbpredict)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6964368955408561

In [19]:
train.unpersist()
train=train.cache()
valid=valid.unpersist()
valid=valid.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ss.stop()